# Working with GDM models

ERAD's `AssetSystem` provides a class method for users working with a GDM `DistributionSystem`. Distribution system components are automatically mapped to the asset models and added to the `AssetSystem`. A hazard simulation can then be set up using the steps listed above. We start by loading a GDM model using the `gdmloader` package. This package can be installed using the command

```shell
pip install gdmloader
```

In [1]:
from gdm.distribution import DistributionSystem
from gdmloader.constants import GCS_CASE_SOURCE
from gdmloader.source import SystemLoader

gdm_loader = SystemLoader()
gdm_loader.add_source(GCS_CASE_SOURCE)

distribution_system: DistributionSystem = gdm_loader.load_dataset(
    source_name=GCS_CASE_SOURCE.name,
    system_type=DistributionSystem,
    dataset_name="p1rhs7_1247",
)
distribution_system.name = "p1rhs7_1247"
distribution_system.info()

/opt/homebrew/Caskroom/miniconda/base/envs/erad/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: Ellipsis is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


System                                
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Property             ┃       Value ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ System name          │ p1rhs7_1247 │
│ Data format version  │       2.0.1 │
│ Components attached  │       13370 │
│ Time Series attached │           0 │
│ Description          │             │
└──────────────────────┴─────────────┘

Component Information                       
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ Type                             ┃ Count ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│ DistributionBus                  │  2510 │
│ DistributionLoad                 │  1896 │
│ DistributionTransformer          │   503 │
│ DistributionTransformerEquipment │    10 │
│ DistributionVoltageSource        │     1 │
│ LoadEquipment                    │  1896 │
│ Location                         │  2510 │
│ MatrixImpedanceBranch            │  1841 │
│ MatrixImpedanceBranchEquipment   │    38 │
│ MatrixImpedanceFuse              │    81 │
│ MatrixImpedanceFuseEquipment     │     6 │
│ MatrixImpedanceSwitch            │    84 │
│ MatrixImpedanceSwitchEquipment   │    15 │
│ PhaseLoadEquipment               │  1948 │
│ PhaseVoltageSourceEquipment      │     3 │
│ TimeCurrentCurve                 │     1 │
│ VoltageLimitSet                  │     8 │
│ VoltageSourceEquipment           │     1 │
│ WindingEquipment                 │    18 │
└──────────────────────────────────┴───────┘

Next, we built the asset system from the gdm `DistributionSystem` using the `from_gdm` method.

In [2]:
from erad.systems import AssetSystem

asset_system = AssetSystem.from_gdm(distribution_system)
asset_system.info()

for a in asset_system.iter_all_components():
    a.pprint()
    break

System                          
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ Property             ┃ Value ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│ System name          │       │
│ Data format version  │       │
│ Components attached  │  4319 │
│ Time Series attached │     0 │
│ Description          │       │
└──────────────────────┴───────┘

Component Information           
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ Type                 ┃ Count ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│ Asset                │  4319 │
└──────────────────────┴───────┘

Asset(
    name='source',
    distribution_asset=UUID('13323a05-a998-490f-8fc3-cf87c23cd95b'),
    asset_type=<AssetTypes.substation: 0>,
    height=<Quantity(3, 'meter')>,
    latitude=38.64060097099336,
    longitude=-122.50688851789585,
    asset_state=[],
    elevation=<Quantity(467.0, 'meter')>
)

### Plotting an AssetSystem

In [3]:
import plotly.graph_objects as go

asset_gdf = asset_system.to_gdf()

fig = go.Figure()
fig.add_trace(
    go.Scattermap(
        lat=asset_gdf["latitude"],
        lon=asset_gdf["longitude"],
        mode="markers",
        marker=dict(size=10, color="blue"),
        opacity=0.4,
        hovertemplate="""
<b>
%{customdata[0]}
</b>
<br>
    type: %{customdata[1]}
<br>
    height: %{customdata[2]}
<br>
    latitude: %{customdata[3]}
<br>
    longitude: %{customdata[4]}
<extra>
</extra>
        """,
        customdata=asset_gdf[
            ["name", "type", "height", "latitude", "longitude"]
        ].values,  # custom data array for hovertemplate
    )
)

# Use OpenStreetMap basemap (no token needed)
fig.update_layout(
    mapbox_style="open-street-map",
    margin={"r": 0, "t": 0, "l": 0, "b": 0},
    mapbox=dict(
        center=dict(
            lat=asset_gdf.latitude.mean(), lon=asset_gdf.longitude.mean()
        ),  # <- Center of your map
        zoom=11,  # <- Zoom level
        style="open-street-map",
    ),
)

fig.show()

### Building a HazardModel

In this section, we built a hazard model and apply the model the asset system.

In [4]:
from datetime import datetime

from shapely.geometry import Polygon
from gdm.quantities import Distance

from erad.models.hazard import FloodModelArea, FloodModel
from erad.systems import HazardSystem
from erad.quantities import Speed

flood_area = FloodModelArea(
    affected_area=Polygon(
        [
            (-122.38, 38.70),
            (-122.35, 38.68),
            (-122.343, 38.69),
            (-122.37, 38.7035),
        ]
    ),
    water_velocity=Speed(0, "meter/second"),
    water_elevation=Distance(160, "meter"),
)

flood = FloodModel(
    name="flood 1",
    timestamp=datetime.now(),
    affected_areas=[flood_area],
)

user_defined_flood_event = HazardSystem(auto_add_composed_components=True)
user_defined_flood_event.add_component(flood)

### Overlaying the HazardModel

We can overlay the hazard model on the same plot using the `add_trace` method. The `show` method can be used to render the image again.

In [5]:
polygon = flood.affected_areas[0].affected_area
lon, lat = polygon.exterior.xy  # returns x and y sequences

fig.add_trace(
    go.Scattermap(
        fill="toself",
        lon=lon.tolist(),
        lat=lat.tolist(),
        marker={"size": 10, "color": flood.affected_areas[0].water_velocity.magnitude},
    )
)

fig.show()

Finally, we can run the actual simulation using the `HarzardSimulator` class from `erad.runner`.

In [6]:
from erad.runner import HarzardSimulator

user_defined_flood_event.info()

hazard_scenario = HarzardSimulator(asset_system=asset_system)
hazard_scenario.run(hazard_system=user_defined_flood_event)

System                          
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ Property             ┃ Value ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│ System name          │       │
│ Data format version  │       │
│ Components attached  │     2 │
│ Time Series attached │     0 │
│ Description          │       │
└──────────────────────┴───────┘

Component Information           
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ Type                 ┃ Count ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│ FloodModel           │     1 │
│ FloodModelArea       │     1 │
└──────────────────────┴───────┘

2025-05-09 15:36:54.529 | WARNING  | erad.runner:run:36 - No HazardFragilityCurves definations found in the passed HazardSystem using default curve definations


Once the simulation is complete, we can visualize the results by plotting the `survival_prob` from the updated gdf dataframe.

In [ ]:
gdf = asset_system.to_gdf()

fig = go.Figure()
fig.add_trace(
    go.Scattermap(
        lat=gdf["latitude"],
        lon=gdf["longitude"],
        mode="markers",
        marker=dict(
            size=10,
            color=gdf["survival_prob"],
            colorbar=dict(title="Survival\nprobability", ticks="outside"),
        ),
        opacity=0.4,
        hovertemplate="""
<b>
%{customdata[0]}
</b>
<br>
    type: %{customdata[1]}
<br>
    height: %{customdata[2]}
<br>
    latitude: %{customdata[3]}
<br>
    longitude: %{customdata[4]}
<br>
    survival prob: %{customdata[5]}
<extra>
</extra>
        """,
        customdata=gdf[
            ["name", "type", "height", "latitude", "longitude", "survival_prob"]
        ].values,  # custom data array for hovertemplate
    )
)

# Use OpenStreetMap basemap (no token needed)
fig.update_layout(
    mapbox_style="open-street-map",
    margin={"r": 0, "t": 0, "l": 0, "b": 0},
    mapbox=dict(
        center=dict(lat=gdf.latitude.mean(), lon=gdf.longitude.mean()),  # <- Center of your map
        zoom=11,  # <- Zoom level
        style="open-street-map",
    ),
)